# Library

In [2]:
!pip install peft ipywidgets bitsandbytes unsloth "xformers==0.0.28.post2" trl
!pip install -U transformers

  Using cached xformers-0.0.28.post2.tar.gz (7.8 MB)


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\USER\\AppData\\Local\\Temp\\pip-install-qly0qb5y\\xformers_6ba84918c29f41feb5416b84202ef134\\third_party\\flash-attention\\csrc\\composable_kernel\\client_example\\24_grouped_conv_activation\\grouped_convnd_bwd_data_bilinear\\grouped_conv_bwd_data_bilinear_residual_fp16.cpp'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths



^C


   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   ---------------------------------------- 10.0/10.0 MB 89.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.2
    Uninstalling transformers-4.45.2:
      Successfully uninstalled transformers-4.45.2


In [9]:
import numpy as np
import asyncio
import aiohttp
import nest_asyncio
import matplotlib.pyplot as plt
from textwrap import wrap
from PIL import Image
import io
from io import BytesIO

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.models as models
from torchvision import datasets, transforms
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoProcessor,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig,
)
from peft import (
    get_peft_model,
    AutoPeftModel,
    AutoPeftModelForCausalLM,
    PeftModel,
    LoraConfig,
    LoftQConfig,
    TaskType,
)
from trl import SFTTrainer
from transformers import (
    TrainingArguments,
    DataCollatorForSeq2Seq,
)
# from unsloth import is_bfloat16_supported
# from unsloth import FastLanguageModel
# from unsloth.chat_templates import (
#     get_chat_template,
#     train_on_responses_only,
#     standardize_sharegpt,
# )
from evaluate import load
from huggingface_hub import notebook_login

# Image Captioning

paper: [Show and Tell: A Neural Image Caption Generator](https://arxiv.org/pdf/1411.4555)

<img src="https://velog.velcdn.com/images%2Fa01152a%2Fpost%2Fc2d815ef-7906-42df-a631-7a5c70e63433%2Fimage.png" width=500>

주어진 이미지의 시각적 내용을 분석해 자연어로 설명하는 기술. <br>
-> 컴퓨터 비전과 자연어 처리가 결합된 작업으로, 이미지를 입력으로 받아 그 이미지에 대한 텍스트 설명을 생성 <br>
주어진 이미지 내에서 feature 추출을 위해 pre-trained 이미지 모델이 사용됨.

이미지 내 객체 인식, 객체의 속성 파악, 이미지 속 상황 이해 등이 요구됨

## Practice

### Multi modal LLM

#### llama

In [5]:
notebook_login()

In [27]:
model_name = 'meta-llama/Llama-3.2-1B-Instruct'
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [28]:
dataset = load_dataset('UCLNLP/adversarial_qa', 'adversarialQA')

In [29]:
train_data = dataset['train']
valid_data = dataset['validation']
test_data = dataset['test']

In [30]:
def formatting(sample):
    eos_token = '<end_of_text>'
    prompt = '''
    ### Question
    {}

    ### Context:
    {}

    ### Answer:
    {}
    '''

    question = sample['question']
    context = sample['context']
    answer = sample['answers']['text'][0]

    prompt = prompt.format(question, context, answer)
    text = ''.join([prompt, eos_token])

    return {'text' : text}

In [31]:
train_data = train_data.map(formatting)
valid_data = valid_data.map(formatting)

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [25]:
train_data['text'][0]

['\n    ### Question\n    What sare the benifts of the blood brain barrir?\n\n    ### Context:\n    Another approach to brain function is to examine the consequences of damage to specific brain areas. Even though it is protected by the skull and meninges, surrounded by cerebrospinal fluid, and isolated from the bloodstream by the blood–brain barrier, the delicate nature of the brain makes it vulnerable to numerous diseases and several types of damage. In humans, the effects of strokes and other types of brain damage have been a key source of information about brain function. Because there is no ability to experimentally control the nature of the damage, however, this information is often difficult to interpret. In animal studies, most commonly involving rats, it is possible to use electrodes or locally injected chemicals to produce precise patterns of damage and then examine the consequences for behavior.\n\n    ### Answer:\n    isolated from the bloodstream\n    <end_of_text>']

In [33]:
training_args = TrainingArguments(
    output_dir='./results/1B',
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    save_steps=50,
    logging_steps=10,
    learning_rate=2e-4,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=valid_data,
    dataset_text_field='text',
    tokenizer=tokenizer
)

c:\Users\USER\.conda\envs\nlp\lib\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
c:\Users\USER\.conda\envs\nlp\lib\site-packages\trl\trainer\sft_trainer.py:149: UserWarning: No `SFTConfig` passed, using `output_dir=tmp_trainer`.
  warnings.warn(f"No `SFTConfig` passed, using `output_dir={output_dir}`.")
c:\Users\USER\.conda\envs\nlp\lib\site-packages\trl\trainer\sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
c:\Users\USER\.conda\envs\nlp\lib\site-packages\trl\trainer\sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTCon

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [34]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: jinhyo97 (jinhyo97-sk-networks). Use `wandb login --relogin` to force relogin


  0%|          | 0/11250 [00:00<?, ?it/s]

AttributeError: 'LlamaForCausalLM' object has no attribute 'loss_function'

In [56]:
from sklearn.model_selection import train_test_split

In [93]:
dataset = load_dataset('beomi/KoAlpaca-RealQA')

In [82]:
dataset

DatasetDict({
    train: Dataset({
        features: ['custom_id', 'question', 'answer'],
        num_rows: 18524
    })
})

In [97]:
dataset = dataset['train'].train_test_split(test_size=0.2)

In [98]:
train_data = dataset['train']
valid_data = dataset['test']

In [100]:
train_data['question']

['교육 및 직업 훈련의 성과를 평가하고 개선하는 방법은 무엇인가요?',
 "기사 요약해줘 '월스트리터저널(WSJ)은 자체 입수한 바이낸스 자료를 검토한 결과 금년 5월 바이낸스의 국가별 거래량은 중국, 한국, 터키, 베트남, 영국령 버진 아일랜드 순이라고 보도했다. \\\nWSJ에 따르면 중국 정부가 2021년 암호화폐 트레이딩을 불법으로 규정했음에도 지난 5월 중국에서 바이낸스 사용자들은 900억 달러의 암호화폐 관련 자산을 거래했다. 이는 바이낸스 전세계 거래량의 20%에 해당된다. \\\n중국에서 지난 5월 바이낸스의 선물 거래는 806억 달러, 현물 거래는 94억 달러로 기록됐다. 같은 기간 한국에서의 선물 거래는 569억3000만 달러, 현물 거래는 13억9000만 달러로 집계됐다.'",
 '안산시 상록구 사동의 오늘 날씨를 알려줘',
 '딥러닝을 통해 컴퓨터가 숫자의 크기를 비교하는 방법',
 '해군의 국제 협력과 연합 작전은 어떻게 이루어지고 있나요?',
 '천국에서의 사회적 구조는 어떻게 형성되어 있습니까?',
 '아이 둘을 포함한 4 가족 부산여행 스케쥴을 추천해줘. 숙소는',
 '주방 가전제품 중에서 가장 에너지 효율이 좋은 것은 뭐예요?',
 '한국 소설 중 2000년 이전의 작품과 2000년 이후의 작품의 차이가 어떤 게 있을까? ',
 '고양이를 위한 인기 있는 장난감 종류는 무엇인가요?',
 '한국현대문학이라는 용어의 정확한 의미는?',
 "질문: '수신자 수신자참조 (경유) 제 목 개발 제한구역 내 건축물 사용승인 사항 통보 우리시 개발재한구역내 건축물 사용승인 사항을 다음과 같이 알려 드리니 관련업무에 참고 하시기 바랍니다. 다 음 = 1 건축물 위치 마산시 내서읍 2 건축주 주소 창원시 용호동 성명 주민등륵번호 3. 허가 번호 및 일자 : 2006-도시 계획 과-신축허가-제19호(2006.07.31) 4 . 사용승인 번호 및 일자 : 제 4 호(2010.02.23) 5. 건축물 현황 단위'\n여기서 제목 부분을 알려줘.\n답변:

In [101]:
def formatting(sample):
    eos_token = '<end_of_text>'
    prompt = '''
    ### Question
    {}

    ### Answer:
    {}
    '''

    question = sample['question']
    answer = sample['answer']

    prompt = prompt.format(question, answer)
    text = ''.join([prompt, eos_token])

    return {'text' : text}

In [102]:
train_data = train_data.map(formatting) 
valid_data = valid_data.map(formatting)

Map:   0%|          | 0/14819 [00:00<?, ? examples/s]

Map:   0%|          | 0/3705 [00:00<?, ? examples/s]

In [103]:
train_data['text'][0]

'\n    ### Question\n    교육 및 직업 훈련의 성과를 평가하고 개선하는 방법은 무엇인가요?\n\n    ### Answer:\n    교육 및 직업 훈련의 성과를 평가하고 개선하는 방법은 다양합니다. 다음은 몇 가지 주요 방법입니다:\n\n1. **성과 지표 설정**: 구체적이고 측정 가능한 목표와 지표를 설정하여 학습 및 훈련의 성과를 평가할 수 있도록 합니다. 예를 들어, 수료율, 시험 점수, 취업률 등이 포함될 수 있습니다.\n\n2. **피드백 수집**: 학습자와 강사로부터 정기적으로 피드백을 받습니다. 설문조사, 인터뷰, 포커스 그룹 등을 통해 훈련의 장단점을 파악하고 개선점을 도출할 수 있습니다.\n\n3. **평가 도구 활용**: 객관적인 평가 도구나 시험을 통해 학습자들의 지식과 기술 습득 여부를 평가합니다. 이를 통해 교육 콘텐츠의 효과성을 측정할 수 있습니다.\n\n4. **데이터 분석**: 학습자의 진행 상황과 성과 데이터를 분석하여 특정 패턴이나 문제점을 파악합니다. 이를 통해 맞춤형 교육 방법을 개발할 수 있습니다.\n\n5. **실습과 현장 경험**: 이론 교육과 함께 실습 및 현장 경험을 강화하여 학습자들이 직업에서 직면할 수 있는 실제 상황에 대비할 수 있도록 합니다.\n\n6. **프로그램 리뷰 및 업데이트**: 정기적으로 교육 및 훈련 프로그램을 검토하고 최신 산업 동향 및 기술을 반영하여 교육 과정을 수정합니다.\n\n7. **멘토링 및 코칭**: 개인화된 멘토링이나 코칭 프로그램을 도입하여 학습자들이 더욱 효율적으로 목표를 달성할 수 있도록 지원합니다.\n\n8. **역량 기반 교육**: 특정 직무에 필요한 역량을 중심으로 교육 과정을 설계하여 실제 업무와의 연결성을 강화합니다.\n\n이러한 방법들을 통해 교육 및 직업 훈련의 효과성을 높이고, 학습자들이 보다 성공적인 직업 활동을 할 수 있도록 지원할 수 있습니다.\n    <end_of_text>'

In [104]:
training_args = TrainingArguments(
    output_dir='./results/1B',
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    save_steps=50,
    logging_steps=10,
    learning_rate=2e-4,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=valid_data,
    dataset_text_field='text',
    tokenizer=tokenizer
)

c:\Users\USER\.conda\envs\nlp\lib\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
c:\Users\USER\.conda\envs\nlp\lib\site-packages\trl\trainer\sft_trainer.py:149: UserWarning: No `SFTConfig` passed, using `output_dir=tmp_trainer`.
  warnings.warn(f"No `SFTConfig` passed, using `output_dir={output_dir}`.")
c:\Users\USER\.conda\envs\nlp\lib\site-packages\trl\trainer\sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
c:\Users\USER\.conda\envs\nlp\lib\site-packages\trl\trainer\sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTCon

Map:   0%|          | 0/14819 [00:00<?, ? examples/s]

Map:   0%|          | 0/3705 [00:00<?, ? examples/s]

In [105]:
trainer.train()

  0%|          | 0/5559 [00:00<?, ?it/s]

AttributeError: 'LlamaForCausalLM' object has no attribute 'loss_function'

## LightWeight

In [108]:
quantization_configs = BitsAndBytesConfig(
    load_in_4bit=True,
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # quantization_configs = quantization_configs
)

lora_config = LoraConfig(
    r=8,
    lora_alpha=5,
    lora_dropout=0.1,
    task_type=TaskType.CAUSAL_LM,
    target_modules=['q_proj', 'k_proj']
)

model = get_peft_model(base_model, lora_config)

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
